# Assortment Optimization

This was an assignment from my Operations Analytics course. The use case here is Designing a Sushi Menu.

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

## Designing a Sushi Menu

### Part 1: Understanding the data

**(a)** The five most preferred sushis for customer 1:
1. 38 Negi Toro
2. 4 Ira
3. 3 Maguro
4. 16 Samon
5. 44 Inari

**(b)** The five least preferred sushis for customer 2:
1. 94 Karasumi
2. 50 Komochi Konbu
3. 93 Kyabia
4. 15 Awabi
5. 17 Kazunoko

**(c)** The five with the best average rank over the 500 customers:
1. 38 Negi Toro
2. 3  Maguro
3. 80 Kurumaebi
4. 62 Negi Toro Maki
5. 1  Ebi

**(d)** The five with the worst average rank over the 500 customers:
1. 93 Kyabia
2. 87 Himo Kyu Maki
3. 94 Karasumi
4. 70 Okura
5. 98 Namako

**(e)** The top five most controversial sushis:
1. 75 Shiso Maki
2. 39 Nattou Maki
3. 5 Uni
4. 30 Kappa Maki
5. 15 Awabi

In [16]:
# Load the sushi utilities matrix
sushi_utilities_matrix = pd.read_csv("sushi_utilities_mat.csv", header = None)
sushi_utilities_matrix = np.asarray(sushi_utilities_matrix)

# Get the number of sushi types
nSushi = sushi_utilities_matrix.shape[1]

print("Shape: ", sushi_utilities_matrix.shape)

# Check sushi 1 
# print("Sushi 1 utilities: ")
# print(sushi_utilities_matrix[:,0])

# Check customer 1
# print("Customer 1 utilities: ")
# print(sushi_utilities_matrix[0,:])

Shape:  (500, 100)


In [17]:
# Load the sushi descriptions
sushi_descriptions = pd.read_csv("sushi_descriptions.csv", header = None)

In [18]:
customer_1_utilities = np.argsort(sushi_utilities_matrix[0,:])
customer_1_utilities = np.flip(customer_1_utilities)
for i in customer_1_utilities[:5]:
    print(sushi_descriptions.iloc[i])

0                                                   38
1    negi_toro (fatty flesh of tuna minced to a pas...
Name: 37, dtype: object
0              4
1    ika (squid)
Name: 3, dtype: object
0                3
1    maguro (tuna)
Name: 2, dtype: object
0                16
1    samon (salmon)
Name: 15, dtype: object
0                            44
1    inari (fried tofu wrapper)
Name: 43, dtype: object


In [19]:
customer_2_utilities = np.argsort(sushi_utilities_matrix[1,:])
for i in customer_2_utilities[:5]:
    print(sushi_descriptions.iloc[i])

0                             94
1    karasumi (dried mullet roe)
Name: 93, dtype: object
0                                          50
1    komochi_konbu (herring roe & sea tangle)
Name: 49, dtype: object
0                 93
1    kyabia (caviar)
Name: 92, dtype: object
0                 15
1    awabi (abalone)
Name: 14, dtype: object
0                        17
1    kazunoko (herring roe)
Name: 16, dtype: object


In [20]:
# Test

temp = np.array([4.8, 2.3, 5.3, 3.9, 1.4, 5.1])
# temp contains the (hypothetical) utilities of 6 products
# Apply argsort once:
temp2 = np.argsort(temp) # What does temp2 contain?
temp3 = np.argsort(temp2) # What does temp3 contain?

In [21]:
temp2  # sort products from low to high utility, display their indices

array([4, 1, 3, 0, 5, 2])

In [22]:
temp3 # sort indices from low to high, display the product utility ranks

array([3, 1, 5, 2, 0, 4])

In [23]:
# Store sushi ranks by customer

nCustomers = sushi_utilities_matrix.shape[0]
sushi_ranks_list = [[] for x in range(nSushi)]

for cust in range(nCustomers):   # loop through 500 customers 
    customer_utilities = np.argsort(sushi_utilities_matrix[cust,:])  # sort the 100 sushi for each customer from low to high utility
    sushi_ranks = np.argsort(customer_utilities)  # the higher the rank, more preferred the sushi
    for s in range(nSushi):
        sushi_ranks_list[s].append(sushi_ranks[s])

In [24]:
# Compute average ranks for sushi

average_ranks = []
for ranks in sushi_ranks_list:
    average_rank = sum(ranks)/len(ranks)
    average_ranks.append(average_rank)

In [25]:
# Sort and find top 5

sorted_sushi_list = np.argsort(average_ranks)
reversed_sushi_list = np.flip(sorted_sushi_list)
top_5_sushi_list = reversed_sushi_list[:5]

for i in top_5_sushi_list:
    print(sushi_descriptions.iloc[i])

0                                                   38
1    negi_toro (fatty flesh of tuna minced to a pas...
Name: 37, dtype: object
0                3
1    maguro (tuna)
Name: 2, dtype: object
0                   80
1    kurumaebi (prawn)
Name: 79, dtype: object
0                                      62
1    negi_toro_maki (roll style of no.37)
Name: 61, dtype: object
0               1
1    ebi (shrimp)
Name: 0, dtype: object


In [26]:
# Sort and find worst 5

worst_5_sushi_list = sorted_sushi_list[:5]
worst_5_sushi_list

for i in worst_5_sushi_list:
    print(sushi_descriptions.iloc[i])

0                 93
1    kyabia (caviar)
Name: 92, dtype: object
0                                              87
1    himo_kyu_maki (part of clam & cucumber roll)
Name: 86, dtype: object
0                             94
1    karasumi (dried mullet roe)
Name: 93, dtype: object
0               70
1    okura (gumbo)
Name: 69, dtype: object
0                       98
1    namako (sea cucumber)
Name: 97, dtype: object


In [27]:
# Compute standard deviation

std_ranks = []
for ranks in sushi_ranks_list:
    mean = sum(ranks)/len(ranks)
    variance = sum([((x - mean) ** 2) for x in ranks]) / len(ranks)
    std = variance ** 0.5
    std_ranks.append(std)

In [28]:
# Find controversial 5

controversial_sushi_list = np.argsort(std_ranks)
top_5_controversial_sushi_list = controversial_sushi_list[-5:] 

for i in top_5_controversial_sushi_list:
    print(sushi_descriptions.iloc[i])

0                                75
1    shiso_maki (perilla leaf roll)
Name: 74, dtype: object
0                                   39
1    nattou_maki (fermented bean roll)
Name: 38, dtype: object
0                   5
1    uni (sea urchin)
Name: 4, dtype: object
0                            30
1    kappa_maki (cucumber roll)
Name: 29, dtype: object
0                 15
1    awabi (abalone)
Name: 14, dtype: object


### Part 2: Common-sense solutions

**(a)** Suppose that we simply offer all sushi products, that is, we set $ S = \{1, 2, ... , n\} $. The expected revenue is **$ 21.51** per customer.

**(b)** The ten highest revenue sushis:
1. 42 Botanebi
2. 21 Hirame
3. 47 Suzuki
4. 20 Chu Toro
5. 80 Kurumaebi
6. 5 Uni
7. 13 Akagai
8. 48 Tarabagani
9. 15 Awabi
10. 9 Toro

The expected revenue is **$ 25.64** per customer.

**(c)** If you offer all options to customers, customers are free to whichever one they like the best and will certainly choose the one that brings them the greatest utilities. Hence, offering all options brings in the same profit as if offering only the options that are most preferred.

The expected revenue in this case is **$ 21.51** per customer.

**(d)** Offering all options or those most preferred by everyone in (a) may include items that are priced relatively low. This set offers great accessibility for customers and allows them to choose whichever they like the best. In other words, it does not force customers to choose their next preferred options that are priced relatively higher. Thus, the expected revenue from this strategy is not optimal.

**(e)** Offering the ten options that are priced the highest in (b) may include items that could turn away customers. Customers may not have a high utility on those high-priced items, so they end up choosing to not purchase. But if including some options with relatively lower priced, they would purchase bringing in revenue.

In [29]:
# Load the sushi price

sushi_info = pd.read_csv("sushi_info.csv", header = None)
prices = np.asarray(sushi_info.iloc[:,2])

In [30]:
# To get the no-purchase utilities, take the max along the product dimension
nopurchase_utilities = np.ones(nCustomers) * 3

# Finally, let us tack these no-purchase utilities to the main utility matrix
utilities_mat = np.concatenate( (sushi_utilities_matrix, nopurchase_utilities[:,None]) , axis = 1)

In [31]:
# Check that this worked
print(utilities_mat.shape)
print(nopurchase_utilities[0])
print(utilities_mat[0, 95:101])

(500, 101)
3.0
[2.14576315 2.03017328 1.36101624 1.6240616  1.54830768 3.        ]


In [32]:
# Function to compute expected revenue
# S - the sushi line, represented as an array of integers
# Precondition: S is an array of integers containing only numbers between 0 and 99, with no repetitions

def expected_revenue(S):
    # Add the no-purchase option:
    # The products are numbered from 0 to 99. Index 100 ( = nProducts) will correspond to the no-purchase option
    S2 = np.append(S, nSushi)
    
    choice_probabilities = {}
    for i in S2:
        choice_probabilities[i] = 0.0;
    
    for cust in range(nCustomers):
        ind = np.argmax( utilities_mat[ cust, S2])  # find the choice with the highest utility for each customer
        choice_probabilities[ S2[ind] ] += 1.0/nCustomers

    exp_revenue = sum( [choice_probabilities[i] * prices[i] for i in S])
        
    return exp_revenue, choice_probabilities


# Let's see one simple example -- we only offer product 1
exp_revenue, choice_probabilities = expected_revenue( [0] )
print("Expected revenue: ", exp_revenue)
print("Choice probabilities: ", choice_probabilities)
print()

# Another example: we offer 1,2,3,4,5
exp_revenue, choice_probabilities = expected_revenue( [1, 2, 3, 4, 5] )
print("Expected revenue: ", exp_revenue)
print("Choice probabilities: ", choice_probabilities)

Expected revenue:  10.708880000000008
Choice probabilities:  {0: 0.7280000000000005, 100: 0.2720000000000002}

Expected revenue:  13.91756000000001
Choice probabilities:  {1: 0.06600000000000004, 2: 0.5660000000000004, 3: 0.09400000000000007, 4: 0.11800000000000009, 5: 0.012, 100: 0.1440000000000001}


In [33]:
# Offer all

S_all = np.array([i for i in range(nSushi)])
exp_revenue_all, choice_probabilities_all = expected_revenue( S_all )
print("Expected per-customer revenue (offer everything): ", exp_revenue_all)

Expected per-customer revenue (offer everything):  21.509720000000016


In [34]:
# Top 10 priced

for i in np.argsort(prices)[-10:]:
    print(sushi_descriptions.iloc[i])

0                         42
1    botanebi (BOTAN shrimp)
Name: 41, dtype: object
0                   21
1    hirame (flatfish)
Name: 20, dtype: object
0                   47
1    suzuki (sea bass)
Name: 46, dtype: object
0                              20
1    chu_toro (mildly_fatty tuna)
Name: 19, dtype: object
0                   80
1    kurumaebi (prawn)
Name: 79, dtype: object
0                   5
1    uni (sea urchin)
Name: 4, dtype: object
0                    13
1    akagai (ark shell)
Name: 12, dtype: object
0                        48
1    tarabagani (king crab)
Name: 47, dtype: object
0                 15
1    awabi (abalone)
Name: 14, dtype: object
0                    9
1    toro (fatty tuna)
Name: 8, dtype: object


In [35]:
# Offer top 10 priced

S_10highestprice = np.argsort(prices)[-10:]
exp_revenue_10highestprice, choice_probabilities_10highestprice = expected_revenue( S_10highestprice )
print("Expected per-customer revenue (offer ten highest price): ", exp_revenue_10highestprice)

Expected per-customer revenue (offer ten highest price):  25.639440000000018


In [36]:
# Use numpy's function argmax to obtain the element with the highest value
# Note: 0:nProducts will leave out column nProducts (= 100, which is where we are storing the no-purchase option)

# Offer most preferred

S_mostpref = np.argmax(utilities_mat[:, 0:nSushi], axis = 1)

S_mostpref = np.unique(S_mostpref)
print(S_mostpref)

exp_revenue_mostpref, choice_probabilities_mostpref = expected_revenue( S_mostpref )
print("Expected per-customer revenue (offer most preferred): ", exp_revenue_mostpref)

[ 0  1  2  3  4  5  6  7  8  9 10 20 26 29 37 43 57 61 71 73 79]
Expected per-customer revenue (offer most preferred):  21.509720000000016


### Part 3: An integer optimization model

**(a)** Explain how constraints (1b) – (1e) correctly model the preferences of each customer:
- Constraint (1b) describes that each customer *k* can only choose one item out of *n* sushis.

- Constraint (1c) and (1d) describes that if a sushi item is included in the offering set, a customer can either choose the item and get their corresponding utility, or choose not to get the item because all the items in the offering set is not at least as good as the no-purchase option. Since we want a customer to have the choice to not purchase if need be based on their utilities, the term,  $u_{k,0} (1 - x_i)$, is needed in (1c) to ensure consistency with (1d) when $x_i = 0$.

- Constraint (1e) describes that if a customer chooses an item (i.e., $y_{k,i} = 1$ ), then the item has to be in the offering set (i.e., $x_i = 1$). If the item is not offered (i.e., $x_i = 0$), then the customer does not choose the item (i.e., $y_{k,i} = 0$).

**(b)** The optimal objective value of the linear optimization relaxation of the LP problem is **$30.59**.

**(c)** An expected per-customer revenue of \\$32 is not possible because the objective value to the relaxation problem is only around \\$30. This value is the upper bound of per-customer revenue from any sushi sets, and no sushi sets could obtain an expected per-customer revenue above that.

**(d)** The expected per-customer revenue of the optimal assortment in the integer version of the LP problem is **$26.24**. This is 22% higher than expected revenue from offering the most preferred sushi set, and 2% higher than expected revenue from offering the 10 highest priced sushis.

**(e)** The restaurant should offer this optimal set of sushis:
1. 9 Toro
2. 15 Awabi
3. 19 Saba
4. 48 Tarabagani
5. 73 Mentaiko Maki
6. 76 Ika Nattou
7. 88 Tobiuo

In [37]:
m = Model()

# Create the decision variables
x = m.addVars(nSushi, vtype = GRB.BINARY)
y = m.addVars(nCustomers, nSushi+1, vtype = GRB.BINARY)

# Create the constraints:
print("Creating constraints...")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nSushi+1)) == 1)
    for i in range(nSushi):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( utilities_mat[k,j] * y[k,j] for j in range(nSushi+1)) >= utilities_mat[k,i] * x[i] + utilities_mat[k,nSushi]*(1 - x[i]))
    m.addConstr( quicksum( utilities_mat[k,j] * y[k,j] for j in range(nSushi+1)) >= utilities_mat[k,nSushi] )

# Create the objective:
print("Creating objective...")
m.setObjective( 1.0/nCustomers * quicksum(prices[i] * y[k,i] for k in range(nCustomers) for i in range(nSushi)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()


S_optimal = [i for i in range(nSushi) if x[i].x > 0.5]
print("Optimal set of products: ", S_optimal)

optimal_revenue = m.objval
print("Optimal revenue: ",optimal_revenue)

# Verify that optimal profit is correct:
exp_revenue_S_optimal, choice_probabilities_S_optimal = expected_revenue( S_optimal )
print("Optimal revenue (via expected_revenue): ", exp_revenue_S_optimal)

Creating constraints...
Creating objective...
Update completed
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 101000 rows, 50600 columns and 5301000 nonzeros
Model fingerprint: 0x9bef12d7
Variable types: 0 continuous, 50600 integer (50600 binary)
Coefficient statistics:
  Matrix range     [6e-06, 5e+00]
  Objective range  [2e-02, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 2 rows and 35747 columns (presolve time = 5s) ...
Presolve removed 73359 rows and 36639 columns (presolve time = 10s) ...
Presolve removed 73429 rows and 36655 columns
Presolve time: 11.81s
Presolved: 27571 rows, 13945 columns, 403592 nonzeros
Variable types: 0 continuous, 13945 integer (13945 binary)
Found heuristic solution: objective 21.5232800
Deterministic concurrent LP optimizer: primal and dual si

In [38]:
m_relaxed = m.relax()

m_relaxed.optimize()

relaxation_bound = m_relaxed.objval

print("Relaxation bound: ", relaxation_bound)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 101000 rows, 50600 columns and 5301000 nonzeros
Model fingerprint: 0xbaeff99d
Coefficient statistics:
  Matrix range     [6e-06, 5e+00]
  Objective range  [2e-02, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve time: 3.57s
Presolved: 101000 rows, 50600 columns, 5301000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...


Barrier performed 0 iterations in 4.84 seconds (4.59 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex

Solved in 1156 iterations and 4.90 seconds (5.57 work units)
Optimal objective  3.058751154e+01
Relaxation bound:  30.58751153829859


In [39]:
# Relative improvements compare to Part 2

rel_improvement_mostpref = 100*(exp_revenue_S_optimal - exp_revenue_mostpref)/exp_revenue_mostpref
print("Relative improvement with most preferred: ", rel_improvement_mostpref)

rel_improvement_10highestprice = 100*(exp_revenue_S_optimal - exp_revenue_10highestprice)/exp_revenue_10highestprice
print("Relative improvement with 10 highest priced: ", rel_improvement_10highestprice)

Relative improvement with most preferred:  22.01246692193112
Relative improvement with 10 highest priced:  2.3600359446228283


In [40]:
# Optimal sushi set

for i in choice_probabilities_S_optimal.keys():
    print("Sushi " + str(i) + ": " + str(choice_probabilities_S_optimal[i]) )

Sushi 8: 0.6060000000000004
Sushi 14: 0.002
Sushi 18: 0.004
Sushi 47: 0.1580000000000001
Sushi 56: 0.0
Sushi 72: 0.002
Sushi 75: 0.004
Sushi 87: 0.002
Sushi 100: 0.22200000000000017


In [41]:
for i in [8, 14, 18, 47, 72, 75, 87]:
    print(sushi_descriptions.iloc[i])

0                    9
1    toro (fatty tuna)
Name: 8, dtype: object
0                 15
1    awabi (abalone)
Name: 14, dtype: object
0                 19
1    saba (mackerel)
Name: 18, dtype: object
0                        48
1    tarabagani (king crab)
Name: 47, dtype: object
0                                    73
1    mentaiko_maki (chili cod roe roll)
Name: 72, dtype: object
0                                     76
1    ika_nattou (squid & fermented bean)
Name: 75, dtype: object
0                      88
1    tobiuo (flying fish)
Name: 87, dtype: object


In [42]:
# EOF